# 如何使用 Claude 3 和 MongoDB 构建 RAG 系统


本教程实现了一个聊天机器人，被提示扮演风险投资技术分析师的角色。该聊天机器人是一个简单的RAG系统，以一组科技新闻文章作为其知识源。
本笔记本涵盖以下内容：

1. 遵循全面的教程来设置开发环境，从安装必要的库到配置MongoDB数据库。
2. 学习高效的数据处理方法，包括创建向量搜索索引和准备用于数据摄取和查询处理的数据。
3. 了解如何在RAG系统中使用Claude 3模型，基于从数据库检索到的上下文信息生成精确的响应。


您需要以下内容：
- Claude API 密钥
- VoyageAI API 密钥
- Hugging Face 访问令牌

## 步骤1：库安装、数据加载和准备


以下是实现代码中使用的工具和库的简要说明：
- anthropic:  Antronic的官方Python库，可访问最先进的语言模型。该库提供对Claude 3系列模型的访问，这些模型可以理解文本和图像。
- datasets: 该库是Hugging Face生态系统的一部分。通过安装'datasets'，我们可以访问许多预处理的和即用的数据集，这些对于训练和微调机器学习模型或基准测试其性能至关重要。
- pandas: 这个数据科学库提供了强大的数据结构和数据操作、处理和分析方法。
- voyageai: 这是访问VoyageAI套件嵌入模型的官方Python客户端库。
- pymongo: PyMongo是MongoDB的Python工具包。它实现了与MongoDB数据库的交互。

In [ ]:
!pip install pymongo datasets pandas anthropic voyageai

下面的代码片段执行以下步骤：
1. 导入必要的库：
- `os` 用于与操作系统交互，
- `requests` 用于发出HTTP请求，
- `io` 模块中的 `BytesIO` 用于处理内存中的字节对象（如文件），
- `pandas`（as pd）用于数据操作和分析，以及
- `userdata` 从google.colab可以访问存储在Google Colab机密中的环境变量。
2. 函数定义：定义了 `download_and_combine_parquet_files` 函数，有两个参数：
- `parquet_file_urls`：字符串URL列表，每个URL指向包含tech-news-embedding数据集子集合的Parquet文件。
- `hf_token` 是表示Hugging Face授权令牌的字符串。访问令牌可以从 [Hugging Face平台](https://huggingface.co/docs/hub/en/security-tokens#:~:text=To%20create%20an%20access%20token,you%27re%20ready%20to%20go!) 创建或复制
3. 下载和读取Parquet文件：函数遍历parquet_file_urls中的每个URL。对于每个URL，它：
- 使用requests.get方法发出GET请求，传递URL和授权标头。
- 检查响应状态码是否为200（OK），表示请求成功。
- 如果成功，它将响应的内容读入BytesIO对象（以将其作为内存中的文件处理），然后使用pandas.read_parquet从该对象中读取Parquet文件到Pandas DataFrame。
- 将DataFrame追加到all_dataframes列表。
4. 合并DataFrame：在将所有Parquet文件下载并读入DataFrame后，检查确保 `all_dataframes` 不为空。如果有要使用的DataFrame，则使用pd.concat将所有DataFrame连接为单个DataFrame，并设置ignore_index=True以重新索引新的合并DataFrame。这个合并的DataFrame是 `download_and_combine_parquet_files` 函数的整体过程输出。

In [ ]:
import requests
from io import BytesIO
import pandas as pd
from google.colab import userdata


def download_and_combine_parquet_files(parquet_file_urls, hf_token):
    """
    使用给定的Hugging Face令牌从提供的URL下载Parquet文件，
    并返回合并的DataFrame。

    参数:
    - parquet_file_urls: 字符串列表，Parquet文件的URL。
    - hf_token: 字符串，Hugging Face授权令牌。

    返回:
    - combined_df: pandas DataFrame，包含来自所有Parquet文件的合并数据。
    """
    headers = {"Authorization": f"Bearer {hf_token}"}
    all_dataframes = []

    for parquet_file_url in parquet_file_urls:
        response = requests.get(parquet_file_url, headers=headers)
        if response.status_code == 200:
            parquet_bytes = BytesIO(response.content)
            df = pd.read_parquet(parquet_bytes)
            all_dataframes.append(df)
        else:
            print(
                f"从 {parquet_file_url} 下载Parquet文件失败: {response.status_code}"
            )

    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        return combined_df
    else:
        print("没有DataFrame可连接。")
        return None

下面是本教程所需的Parquet文件列表。完整文件列表位于[这里](https://huggingface.co/datasets/MongoDB/tech-news-embeddings/tree/refs%2Fconvert%2Fparquet/default/train)。每个Parquet文件代表大约45,000个数据点。

在下面的代码片段中，tech-news-embeddings数据集的一个子集被分组为单个DataFrame，然后将其分配给变量 `combined_df`。

In [ ]:
# 取消注释下面的链接以加载更多数据
# 完整数据列表请访问: https://huggingface.co/datasets/MongoDB/tech-news-embeddings/tree/refs%2Fconvert%2Fparquet/default/train
parquet_files = [
    "https://huggingface.co/api/datasets/AIatMongoDB/tech-news-embeddings/parquet/default/train/0000.parquet",
    # "https://huggingface.co/api/datasets/AIatMongoDB/tech-news-embeddings/parquet/default/train/0001.parquet",
    # "https://huggingface.co/api/datasets/AIatMongoDB/tech-news-embeddings/parquet/default/train/0002.parquet",
    # "https://huggingface.co/api/datasets/AIatMongoDB/tech-news-embeddings/parquet/default/train/0003.parquet",
    # "https://huggingface.co/api/datasets/AIatMongoDB/tech-news-embeddings/parquet/default/train/0004.parquet",
    # "https://huggingface.co/api/datasets/AIatMongoDB/tech-news-embeddings/parquet/default/train/0005.parquet",
]

hf_token = userdata.get("HF_TOKEN")
combined_df = download_and_combine_parquet_files(parquet_files, hf_token)

作为数据准备的最后阶段，下面的代码片段显示了从分组数据集中删除 `_id` 列的步骤，因为对于本教程的后续步骤来说这是不必要的。此外，每个数据点的embedding列内的数据从numpy数组转换为Python列表，以防止在数据摄取期间发生与不兼容数据类型相关的错误。

In [ ]:
# 从初始数据集中删除_id列
combined_df = combined_df.drop(columns=["_id"])

# 删除初始的嵌入列，因为我们将使用VoyageAI嵌入模型创建新的嵌入
combined_df = combined_df.drop(columns=["embedding"])

In [ ]:
combined_df.head()

In [ ]:
# 由于VoyageAI API的速率限制，将用于此演示的文档数量限制为500
# 更多关于VoyageAI速率限制的信息请访问: https://docs.voyageai.com/docs/rate-limits
max_documents = 500

if len(combined_df) > max_documents:
    combined_df = combined_df[:max_documents]

In [ ]:
import voyageai

vo = voyageai.Client(api_key=userdata.get("VOYAGE_API_KEY"))


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("尝试为空文本获取嵌入向量。")
        return []

    embedding = vo.embed(text, model="voyage-large-2", input_type="document")

    return embedding.embeddings[0]


combined_df["embedding"] = combined_df["description"].apply(get_embedding)

combined_df.head()

## 步骤2：数据库和集合创建

**要创建新的MongoDB数据库，请设置数据库集群：**
1. 注册一个[免费的MongoDB Atlas账户](https://www.mongodb.com/cloud/atlas/register?utm_campaign=devrel&utm_source=community&utm_medium=cta&utm_content=Partner%20Cookbook&utm_term=richmond.alake)，或对于现有用户，[登录MongoDB Atlas](https://account.mongodb.com/account/login?utm_campaign=devrel&utm_source=community&utm_medium=cta&utm_content=Partner%20Cookbook&utm_term=richmond.alake)
2. 选择左侧窗格中的"Database"选项，这将导航到数据库部署页面，显示任何现有集群的部署规范。通过点击"+Create"按钮创建新的数据库集群。
3. 有关数据库集群设置和获取URI的帮助，请参考我们的MongoDB集群设置指南和获取连接字符串指南。
注意：在创建概念证明时，不要忘记为Python主机或任何IP的0.0.0.0/0列入白名单。
4. 成功创建和部署集群后，集群在'数据库部署'页面上变得可访问。
5. 点击集群的"Connect"按钮，查看通过各种语言驱动程序设置与集群连接的选项。
6. 本教程只需要集群的URI（唯一资源标识符）。获取URI并将其复制到Google Colab机密环境中，变量名为MONGO_URI，或将其放置在.env文件或等效文件中。


一旦您创建了集群，导航到集群页面并通过单击+创建数据库在MongoDB Atlas集群内创建数据库和集合。
数据库将命名为 `tech_news`，集合将命名为 `hacker_noon_tech_news`。

## 步骤3：向量搜索索引创建

到目前为止，您已经创建了集群、数据库和集合。

本节中的步骤对于确保可以使用输入到聊天机器人中的查询并在hacker_noon_tech_news集合中的记录中进行搜索来执行向量搜索至关重要。本步骤的目标是创建向量搜索索引。要实现这一点，请参考官方的[向量搜索索引创建指南](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-index/)。

在使用MongoDB Atlas上的JSON编辑器创建向量搜索索引时，请确保您的向量搜索索引命名为vector_index，向量搜索索引定义如下：

```
{
 "fields": [{
     "numDimensions": 1536,
     "path": "embedding",
     "similarity": "cosine",
     "type": "vector"
   }]
}

```

## 步骤4：数据摄取

要将数据摄取到前面步骤中创建的MongoDB数据库中。必须执行以下操作：
- 连接到数据库和集合
- 清除集合中的任何现有记录
- 在摄取之前将数据集的Pandas DataFrame转换为字典
- 使用批量操作将字典摄取到MongoDB中

本教程需要集群的URI（唯一资源标识符）。获取URI并将其复制到Google Colab机密环境中，变量名为MONGO_URI，或将其放置在.env文件或等效文件中。

In [ ]:
import pymongo
from google.colab import userdata


def get_mongo_client(mongo_uri):
    """与MongoDB建立连接。"""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("连接MongoDB成功")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"连接失败: {e}")
        return None


mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI未在环境变量中设置")

mongo_client = get_mongo_client(mongo_uri)

DB_NAME = "tech_news"
COLLECTION_NAME = "hacker_noon_tech_news"

db = mongo_client[DB_NAME]
collection = db[COLLECTION_NAME]

In [ ]:
# 确保我们使用的是全新集合
# 删除集合中的任何现有记录
collection.delete_many({})

In [ ]:
# 数据摄取
combined_df_json = combined_df.to_dict(orient="records")
collection.insert_many(combined_df_json)

## 步骤5：向量搜索

本节展示了创建向量搜索自定义函数的过程，该函数接受用户查询，对应聊天机器人的输入。该函数还接受第二个参数 `collection`，它指向包含要在其上执行向量搜索操作的记录的数据库集合。

`vector_search` 函数产生从MongoDB聚合管道中概述的一系列操作派生的向量搜索结果。该管道包括 `$vectorSearch` 和 `$project` 阶段，并基于用户查询的向量嵌入执行查询。然后它格式化结果，省略对后续过程不必要的任何记录属性。

下面的代码片段执行以下操作以允许对电影进行语义搜索：
1. 定义 `vector_search` 函数，该函数接受用户的查询字符串和MongoDB集合作为输入，并返回基于向量相似性搜索匹配查询的文档列表。
2. 通过调用前面定义的函数 `get_embedding` 为用户查询生成嵌入，该函数将查询字符串转换为向量表示。
3. 为MongoDB的聚合函数构造管道，包含两个主要阶段：`$vectorSearch` 和 `$project`。
4. `$vectorSearch` 阶段执行实际的向量搜索。index字段指定要用于向量搜索的向量索引，这应与前面步骤中在向量搜索索引定义中输入的名称相对应。queryVector字段采用用户查询的嵌入表示。path字段对应于包含嵌入的文档字段。`numCandidates` 指定要考虑考虑的候选文档数量，以及要返回的结果数量限制。
5. $project阶段格式化结果以排除_id和`embedding`字段。
6. 聚合执行定义的管道以获得向量搜索结果。最后的操作将数据库返回的游标转换为列表。

In [ ]:
def vector_search(user_query, collection):
    """
    基于用户查询在MongoDB集合中执行向量搜索。

    Args:
    user_query (str): 用户的查询字符串。
    collection (MongoCollection): 要搜索的MongoDB集合。

    Returns:
    list: 匹配文档的列表。
    """

    # 为用户查询生成嵌入
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "无效查询或嵌入生成失败。"

    # 定义向量搜索管道
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # 要考虑的候选匹配数
                "limit": 5,  # 返回前5个匹配
            }
        },
        {
            "$project": {
                "_id": 0,  # 排除_id字段
                "embedding": 0,  # 排除嵌入字段
                "score": {
                    "$meta": "vectorSearchScore"  # 包含搜索分数
                },
            }
        },
    ]

    # 执行搜索
    results = collection.aggregate(pipeline)
    return list(results)

## 步骤6：使用Claude 3模型处理用户查询

教程的最后部分概述了执行的操作序列如下：

- 接受字符串形式的用户查询。
- 利用VoyageAI嵌入模型为用户查询生成嵌入。
- 加载Anthropic Claude 3，特别是'claude-opus-4-1'模型，作为RAG系统的基础模型。
- 使用用户查询的嵌入执行向量搜索，以从知识库中获取相关信息，为基础模型提供额外的上下文。
- 将用户查询和收集到的其他信息都提交给基础模型以生成响应。


一个重要的注意事项是，用户查询嵌入的维度与MongoDB Atlas上向量搜索索引定义中设置的维度相匹配。

本节的下一步是导入anthropic库并加载客户端以访问Anthropic的处理消息和访问Claude模型的方法。确保您从[Anthropic官方网站](https://console.anthropic.com/settings/keys)设置页面获取Claude API密钥。

In [ ]:
import anthropic

client = anthropic.Client(api_key=userdata.get("ANTHROPIC_API_KEY"))

下面是对下面代码片段中操作的更详细描述：

1. 向量搜索执行：函数首先使用用户的查询和指定的集合作为参数调用 `vector_search`。这在集合内执行搜索，利用向量嵌入来查找与查询相关的信息。
2. 编译搜索结果：`search_result` 初始化为空字符串，用于聚合搜索中的信息。通过遍历 `vector_search` 函数返回的结果来编译搜索结果，将每个项目的详细信息（标题、公司名称、URL、发布日期、文章URL和描述）格式化为人类可读的字符串，将此信息追加到search_result，在每个条目的末尾添加换行符\n。
3. 使用Anthropic客户端生成响应：函数然后构造对Claude API的请求（通过客户端对象，可能是前面创建的anthropic.Client类的实例）。它指定：
- 要使用的模型（"claude-opus-4-1"）表示Claude 3模型的特定版本。
- 生成响应的最大令牌限制（max_tokens=1024）。
- 系统描述指导模型表现为"风险投资技术分析师"，可以访问科技公司的文章和信息，使用此上下文提供建议。
- 模型要处理的实际消息将用户查询与聚合搜索结果作为上下文相结合。
4. 返回生成的响应和搜索结果：从响应的第一个内容项中提取并返回响应文本，以及编译的搜索结果。

In [ ]:
def handle_user_query(query, collection):
    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += (
            f"标题: {result.get('title', 'N/A')}, "
            f"公司名称: {result.get('companyName', 'N/A')}, "
            f"公司网址: {result.get('companyUrl', 'N/A')}, "
            f"发布日期: {result.get('published_at', 'N/A')}, "
            f"文章网址: {result.get('url', 'N/A')}, "
            f"描述: {result.get('description', 'N/A')}, \n"
        )

    response = client.messages.create(
        model="claude-opus-4-1",
        max_tokens=1024,
        system="你是风险投资技术分析师，可以访问一些科技公司的文章和信息。你使用给你的信息来提供建议。",
        messages=[
            {
                "role": "user",
                "content": "回答这个用户查询: "
                + query
                + " 包含以下上下文: "
                + search_result,
            }
        ],
    )

    return (response.content[0].text), search_result

本教程的最后一步是初始化查询，将其传递给 `handle_user_query` 函数并打印返回的响应。

In [ ]:
# 执行查询并检索来源
query = "给我最好的科技股投资并告诉我为什么"
response, source_information = handle_user_query(query, collection)

print(f"响应: {response}")
print(f"\n来源信息: \n{source_information}")